In [1]:
import numpy as np
import pandas as pd

import ast
import os

In [2]:
def parse_results(file_name):
    grid_search_results = []
    with open(file_name, 'r') as file:
        for line in file.readlines():
            if line[0:4] == '----':
                grid_search_results.append([])
                continue

            if line[0] == '(':
                fixed_line = '[' + line.strip().replace(') (', '), (') + ']'
                params = ast.literal_eval(fixed_line)
                nested = [list(zip(key.split(','), np.array(val).flatten())) for (key, val) in params]
                unnested = [item for sublist in nested for item in sublist]
                grid_search_results[-1].append({key: val for (key, val) in unnested})
                grid_search_results[-1].append([])
                continue

            result = line

            if result[0] == '[':
                grid_search_results[-1][-1].append(ast.literal_eval(result))
                continue

    grid_search_results = [(params, np.array(values)) for params, values in grid_search_results]
    return grid_search_results

In [117]:
DIR_PATH = '../blueberry/grid_search_results'
res_files = [os.path.join(DIR_PATH, f) for f in [f for f in os.listdir(DIR_PATH) if os.path.isfile(os.path.join(DIR_PATH, f))] if f[0:7] != 'subsets']

In [118]:
print(res_files)

['../blueberry/grid_search_results\\both_mega_50.txt', '../blueberry/grid_search_results\\both_small_100.txt', '../blueberry/grid_search_results\\both_small_500.txt', '../blueberry/grid_search_results\\euc_small_2000.txt', '../blueberry/grid_search_results\\hyp_small_2000.txt', '../blueberry/grid_search_results\\hyp_small_800.txt']


In [119]:
all_res = [parse_results(f) for f in res_files]
all_res = [item for row in all_res for item in row]

In [120]:
all_params = [p for p, _ in all_res]
all_values = [np.array(v) for _, v in all_res]

In [121]:
hyp_indices = [i for i, p in enumerate(all_params) if p['model_type'] == 'hyp']
euc_indices = [i for i, p in enumerate(all_params) if p['model_type'] == 'euc']

In [122]:
hyp_params = []
hyp_values = []

euc_params = []
euc_values = []

In [123]:
for i, (p, v) in enumerate(zip(all_params, all_values)):
    if i in hyp_indices:
        hyp_params.append(p)
        hyp_values.append(v)
    else:
        euc_params.append(p)
        euc_values.append(v)

In [153]:
hyp_size_values = {size: [] for size in set([p['layer_size'] for p in hyp_params])}
euc_size_values = {size: [] for size in set([p['layer_size'] for p in euc_params])}

In [154]:
for i, (p, v) in enumerate(zip(hyp_params, hyp_values)):
    hyp_size_values[p['layer_size']].append(v.mean(axis=0))
for p, v in zip(euc_params, euc_values):
    euc_size_values[p['layer_size']].append(v.mean(axis=0))

In [155]:
min(hyp_size_values[4][177])

0.0293335895

In [133]:
for size in [2,4,8,16,32,64,128,256]:
    # print(f'{size}: {min([v.min() for v in hyp_size_values[size]]):.3}, {min([v[-1] for v in hyp_size_values[size]]):.3}')
    # print(f'{size}: {min([v[-1] for v in hyp_size_values[size]]):.2}')
    print(f'{size}: {min([min(v[0], min(v[49::50])) for v in hyp_size_values[size]]):.2}')

2: 0.033
4: 0.029
8: 0.03
16: 0.029
32: 0.031
64: 0.03
128: 0.038
256: 0.037


In [130]:
# for size in sorted(euc_size_values.keys()):
for size in [2,4,8,16,32,64,128,256]:
    # print(f'{size}: {min([v.min() for v in euc_size_values[size]]):.3}, {min([v[-1] for v in euc_size_values[size]]):.3}')
    print(f'{size}: {min([min(v[0], min(v[49::50])) for v in euc_size_values[size]]):.2}')

2: 0.029
4: 0.029
8: 0.029
16: 0.029
32: 0.029
64: 0.029
128: 0.029
256: 0.029


In [142]:
# [(i, [v.mean(axis=0)[0], *v.mean(axis=0)[49::50]]) for i, v in enumerate(hyp_values)]
[(i, v.mean(axis=0)) for i, v in enumerate(hyp_values)]

[(0,
  array([0.68501879, 0.62228764, 0.58546847, 0.57067146, 0.56583017,
         0.52302163, 0.47450078, 0.43170775, 0.39663324, 0.36813437,
         0.33716436, 0.31533145, 0.30614854, 0.29527999, 0.27787355,
         0.26618534, 0.24816091, 0.2413258 , 0.24167996, 0.23539681,
         0.23165666, 0.22408988, 0.22098894, 0.21477027, 0.21465588,
         0.21278356, 0.21031349, 0.20762569, 0.20831381, 0.20568941,
         0.20403465, 0.20062788, 0.20002388, 0.19816157, 0.1960535 ,
         0.19658599, 0.19491453, 0.19017813, 0.19304221, 0.18899847,
         0.18571754, 0.19010291, 0.18257723, 0.18356091, 0.18361407,
         0.17950632, 0.18010386, 0.17578763, 0.17967079, 0.17394943])),
 (1,
  array([0.68501974, 0.62228912, 0.58547321, 0.5706126 , 0.5658116 ,
         0.52307092, 0.4745509 , 0.43171943, 0.39657847, 0.36810209,
         0.33716428, 0.31524186, 0.30596201, 0.29526886, 0.27785265,
         0.26615601, 0.24823559, 0.24117748, 0.24161494, 0.23534448,
         0.23159697, 

In [150]:
hyp_params[1753]

{'model_type': 'hyp',
 'num_hidden_layers': 5,
 'layer_size': 4,
 'lr': 0.005,
 'weight_decay': 0.005,
 'batch_size': 1024,
 'epochs': 2000,
 'curvature': -1}

In [161]:
hyp_values[1753].mean(axis=0).argmin()
hyp_values[1753].mean(axis=0)[1199]

0.02938354